In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns',False)

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import lightgbm

In [3]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

In [4]:
train.drop(columns=['id','cat10','cat5','cat7','cat8'],inplace=True)
test.drop(columns=['id','cat10','cat5','cat7','cat8'],inplace=True)

In [5]:
cat = [col for col in train.columns if 'cat' in col]
cont = [col for col in test.columns if 'cont' in col]
all_features = cat + cont

In [6]:
oe=OrdinalEncoder()
train[cat]=oe.fit_transform(train[cat])
test[cat]=oe.transform(test[cat])

In [7]:
data=train.drop(columns=['target'])
target=train['target']

In [8]:
def objective(trial , data = data , target = target):
    train_x , test_x , train_y , test_y = train_test_split(data , target , \
            test_size = 0.28059109276941666 , random_state = 42)
    
    params = {
        'reg_alpha' : trial.suggest_loguniform('reg_alpha' , 1e-5 , 10),
        'reg_lambda' : trial.suggest_loguniform('reg_lambda' , 1e-5 , 10),
        'num_leaves' : trial.suggest_int('num_leaves' , 11 , 300),
        'learning_rate' : trial.suggest_uniform('learning_rate' , 0 , 0.1),
        'max_depth' : trial.suggest_int('max_depth' , 5 , 20),
        'n_estimators' : trial.suggest_int('n_estimators' , 1 , 9999),
        'min_child_samples' : trial.suggest_int('min_child_samples' , 1 , 100),
        'min_child_weight' : trial.suggest_loguniform('min_child_weight' , 1e-5 , 1),
        'subsample' : trial.suggest_uniform('subsample' , 0 , 1.0),
        'colsample_bytree' : trial.suggest_loguniform('colsample_bytree' , 1e-5 , 1),
        'random_state' : trial.suggest_categorical('random_state' , [0,42,2021,555]),
        'metric' : 'auc',
        'device_type' : 'gpu',
    }
    model = lightgbm.LGBMClassifier(**params)
    model.fit(train_x , train_y , eval_set = [(test_x , test_y)] , early_stopping_rounds = 200 , \
             verbose = False)
    preds = model.predict_proba(test_x)[:,1]
    auc = roc_auc_score(test_y , preds)
    return auc

In [9]:
study = optuna.create_study(direction = 'maximize' , study_name = 'lgbm')
study.optimize(objective , n_trials = 50)
print('numbers of the finished trials:' , len(study.trials))
print('the best params:' , study.best_trial.params)
print('the best value:' , study.best_value)

[I 2021-03-26 17:20:47,603] A new study created in memory with name: lgbm
[W 2021-03-26 17:20:48,179] Trial 0 failed because of the following error: LightGBMError('GPU Tree Learner was not enabled in this build.\nPlease recompile with CMake option -DUSE_GPU=1')
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/optuna/_optimize.py", line 211, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-8-0acfd937e6f7>", line 22, in objective
    verbose = False)
  File "/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py", line 231, in train
    booster = Booster(params=params, train_set=train_set)
  File "/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py", line 2061, i

LightGBMError: GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1

In [10]:
optuna.visualization.plot_optimization_history(study)

[W 2021-03-26 17:20:48,404] Study instance does not contain trials.


In [11]:
print('the best params:' , study.best_trial.params)

ValueError: No trials are completed yet.

In [12]:
params=study.best_params

ValueError: No trials are completed yet.

In [13]:
params

NameError: name 'params' is not defined

In [14]:
params={'reg_alpha': 0.002886412378348068,
 'reg_lambda': 3.133777439786117e-05,
 'num_leaves': 181,
 'learning_rate': 0.09024790642402873,
 'max_depth': 13,
 'n_estimators': 8328,
 'min_child_samples': 31,
 'min_child_weight': 0.03498506654674707,
 'subsample': 0.332175013858841,
 'colsample_bytree': 0.29882513770591124,
 'random_state': 42}

In [15]:
params_new=study.best_params

ValueError: No trials are completed yet.

In [16]:
params_new

NameError: name 'params_new' is not defined

In [17]:
params_1={'reg_alpha': 0.8652493397013724,
 'reg_lambda': 0.028016961044558177,
 'num_leaves': 286,
 'learning_rate': 0.03376414440763976,
 'max_depth': 13,
 'n_estimators': 5721,
 'min_child_samples': 41,
 'min_child_weight': 0.00032169039200189445,
 'subsample': 0.28929714816950847,
 'colsample_bytree': 0.34162338483795796,
 'random_state': 42}